In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os
from codeStore import support_fun as spf
import importlib
PWD = os.getcwd()

## ecoli near sphere, sphere is fixed 

In [33]:
use_node = 1
njob_node = 1
n_job_pbs = use_node * njob_node
random_order = True
n_job_pbs = 10000


sm, legendre_m, legendre_k, epsilon = 'lg_rs', 3, 2, 3
rh1, rh2, nth, eh, n_tail = 0.5, 0.05, 18, 0, 1
ph, ch = 2, 4
hfct, with_cover, with_T_geo, left_hand = 1, 2, 0, 0
rs1, rs2, ds, es, dist_hs, rel_whz = 2, 1, 0.1, 0, 0.5, 1
centerx, centerz = 5, 0
ellipse_rs1, ellipse_rs2, ellipse_rs3, ellipse_ds, ellipse_es = 3, 3, 3, 0.2, 0
ellipse_ux, ellipse_uy, ellipse_uz, ellipse_wx, ellipse_wy, ellipse_wz = np.zeros(6)
ksp_max_it, ksp_rtol, ksp_atol = 1000, 1e-20, 1e-200

def comm_obj_helicoid_hlx(sm, legendre_m, legendre_k, epsilon, 
                          rh1, rh2, nth, eh, ph, ch, n_tail, 
                          hfct, with_cover, with_T_geo, left_hand, 
                          rs1, rs2, ds, es, dist_hs, rel_whz, 
                          centerx, centery, centerz, 
                          ellipse_rs1, ellipse_rs2, ellipse_rs3, ellipse_ds, ellipse_es, 
                          ellipse_ux, ellipse_uy, ellipse_uz, ellipse_wx, ellipse_wy, ellipse_wz, 
                          ksp_max_it, ksp_rtol, ksp_atol, f, mpinp=24): 
    if 'rs' in sm:
        err_msg = 'the regularized family methods requires eh==0. '
        assert np.isclose(eh, 0), err_msg
        err_msg = 'the regularized family methods requires ellipse_es==0. '
        assert np.isclose(ellipse_es, 0), err_msg
        err_msg = 'the regularized family methods requires es==0. '
        assert np.isclose(es, 0), err_msg
        
    main_fun = 'main_fix_sphere' 
    ts = 'mpirun -np %d python ../../ecoliNearSphere.py -%s 1 ' % (mpinp, main_fun)
    ts = ts + ' -sm %s -legendre_m %d -legendre_k %d -epsilon %f ' % \
                (sm,    legendre_m,    legendre_k,    epsilon)
    ts = ts + ' -rh1 %f -rh2 %f -nth %d -eh %f -ph %f -ch %f -n_tail %d ' % \
                (rh1,    rh2,    nth,    eh,    ph,    ch,    n_tail)
    ts = ts + ' -hfct %f -with_cover %d -with_T_geo %d -left_hand %d ' % \
                (hfct,    with_cover,    with_T_geo,    left_hand)
    ts = ts + ' -rs1 %f -rs2 %f -ds %f -es %f -dist_hs %f -rel_whz %f ' % \
                (rs1,    rs2,    ds,    es,    dist_hs,    rel_whz)
    ts = ts + ' -centerx %f -centery %f -centerz %f ' % \
                (centerx,    centery,    centerz)
    ts = ts + ' -ellipse_rs1 %f -ellipse_rs2 %f -ellipse_rs3 %f -ellipse_ds %f -ellipse_es %f ' % \
                (ellipse_rs1,    ellipse_rs2,    ellipse_rs3,    ellipse_ds,    ellipse_es)
    ts = ts + ' -ellipse_ux %f -ellipse_uy %f -ellipse_uz %f -ellipse_wx %f -ellipse_wy %f -ellipse_wz %f ' % \
                (ellipse_ux,    ellipse_uy,    ellipse_uz,    ellipse_wx,    ellipse_wy,    ellipse_wz)
    ts = ts + ' -ksp_max_it %d -ksp_rtol %e -ksp_atol %e -f %s' % \
                (ksp_max_it,    ksp_rtol,    ksp_atol,    f)
    return ts

# write_pbs_head = spf.write_pbs_head_q03
# use_node = 10
# ds, nth, ellipse_ds = 0.06, 15, 0.1
# centery_list = np.linspace(-15, 15, 121)
# job_dir = 'fix_case_a'

write_pbs_head = spf.write_pbs_head_q03
use_node = 10
ds, nth, ellipse_ds = 0.1, 10, 0.16
centery_list = np.linspace(-15, 15, 31)
job_dir = 'fix_case_b'

###############################################################################################
# generate all comment file. 
comm_list = []
txt_list = []
# helicoid_r_list, ph_list, ch_list = np.meshgrid(helicoid_r_list, ph_list, ch_list)
for centery in centery_list:
    f = 'centery%.4f' % centery
    ts = comm_obj_helicoid_hlx(sm, legendre_m, legendre_k, epsilon, 
                               rh1, rh2, nth, eh, ph, ch, n_tail, 
                               hfct, with_cover, with_T_geo, left_hand, 
                               rs1, rs2, ds, es, dist_hs, rel_whz, 
                               centerx, centery, centerz, 
                               ellipse_rs1, ellipse_rs2, ellipse_rs3, ellipse_ds, ellipse_es, 
                               ellipse_ux, ellipse_uy, ellipse_uz, ellipse_wx, ellipse_wy, ellipse_wz, 
                               ksp_max_it, ksp_rtol, ksp_atol, f)
    comm_list.append(ts)
    txt_list.append(f)
spf.write_main_run_comm_list(comm_list, txt_list, use_node, njob_node, job_dir, 
                             write_pbs_head, n_job_pbs, random_order=random_order)

make folder /home/zhangji/stokes_flow_master/ecoliSphere/database/fix_case_b
input 31 cases.
generate 1 pbs files in total.
 --->>random order mode is ON. 
Command of first case is:
mpirun -np 24 python ../../ecoliNearSphere.py -main_fix_sphere 1  -sm lg_rs -legendre_m 3 -legendre_k 2 -epsilon 3.000000  -rh1 0.500000 -rh2 0.050000 -nth 10 -eh 0.000000 -ph 2.000000 -ch 4.000000 -n_tail 1  -hfct 1.000000 -with_cover 2 -with_T_geo 0 -left_hand 0  -rs1 2.000000 -rs2 1.000000 -ds 0.100000 -es 0.000000 -dist_hs 0.500000 -rel_whz 1.000000  -centerx 5.000000 -centery 8.000000 -centerz 0.000000  -ellipse_rs1 3.000000 -ellipse_rs2 3.000000 -ellipse_rs3 3.000000 -ellipse_ds 0.160000 -ellipse_es 0.000000  -ellipse_ux 0.000000 -ellipse_uy 0.000000 -ellipse_uz 0.000000 -ellipse_wx 0.000000 -ellipse_wy 0.000000 -ellipse_wz 0.000000  -ksp_max_it 1000 -ksp_rtol 1.000000e-20 -ksp_atol 1.000000e-200 -f centery8.0000


True

In [14]:
np.linspace(-15, 15, 61)

array([-15. , -14.5, -14. , -13.5, -13. , -12.5, -12. , -11.5, -11. ,
       -10.5, -10. ,  -9.5,  -9. ,  -8.5,  -8. ,  -7.5,  -7. ,  -6.5,
        -6. ,  -5.5,  -5. ,  -4.5,  -4. ,  -3.5,  -3. ,  -2.5,  -2. ,
        -1.5,  -1. ,  -0.5,   0. ,   0.5,   1. ,   1.5,   2. ,   2.5,
         3. ,   3.5,   4. ,   4.5,   5. ,   5.5,   6. ,   6.5,   7. ,
         7.5,   8. ,   8.5,   9. ,   9.5,  10. ,  10.5,  11. ,  11.5,
        12. ,  12.5,  13. ,  13.5,  14. ,  14.5,  15. ])